# Calculating potential evaporation (Ep)

In [ ]:
import pandas as pd
import rioxarray as rioxr
import xarray as xr
import numpy as np
import pyet
import os

os.chdir('/home/rooda/Dropbox/Patagonia/Data/') 
days = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
period = slice("1980-01-01", "2020-12-31")

encode_pet   = {'PET':  {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999}}

## Data

The variables of each dataset are Tmax, Tmin and Tavg (Hargreaves equation)

In [ ]:
# CR2MET
cr2met_tmax  = xr.open_dataset("Temperature/Tmax_CR2MET_1960_2021d.nc", chunks ="auto").rename({"tmax":"T2M"})["T2M"].sel(time = period)
cr2met_tmin  = xr.open_dataset("Temperature/Tmin_CR2MET_1960_2021d.nc", chunks ="auto").rename({"tmin":"T2M"})["T2M"].sel(time = period)
cr2met_tmean = xr.open_dataset("Temperature/Tavg_CR2MET_1960_2021d.nc", chunks ="auto").rename({"t2m":"T2M"})["T2M"].sel(time = period)
cr2met_lat   = cr2met_tmean.lat * np.pi / 180

# PMET
pmet_tmax  = xr.open_dataset("Temperature/Tmax_PMET_1980_2020d.nc", chunks ="auto").rename({"tmax":"T2M"})["T2M"].sel(time = period)
pmet_tmin  = xr.open_dataset("Temperature/Tmin_PMET_1980_2020d.nc", chunks ="auto").rename({"tmin":"T2M"})["T2M"].sel(time = period)
pmet_tmean = xr.open_dataset("Temperature/Tavg_PMET_1980_2020d.nc", chunks ="auto").rename({"t2m":"T2M"})["T2M"].sel(time = period)
pmet_lat   = pmet_tmean.latitude * np.pi / 180

# ERA5
era5_tmax  = xr.open_dataset("Temperature/Tmax_ERA5_hr_1980_2020d.nc", chunks ="auto").rename({"tmax":"T2M"})["T2M"].sel(time = period)
era5_tmin  = xr.open_dataset("Temperature/Tmin_ERA5_hr_1980_2020d.nc", chunks ="auto").rename({"tmin":"T2M"})["T2M"].sel(time = period)
era5_tmean = xr.open_dataset("Temperature/Tavg_ERA5_hr_1980_2020d.nc", chunks ="auto").rename({"t2m":"T2M"})["T2M"].sel(time = period)
era5_lat   = era5_tmean.latitude * np.pi / 180

# MSWEP/MSWX
mswx_tmax  = xr.open_dataset("Temperature/Tmax_MSWX_1979_2019d.nc", chunks ="auto").rename({"t2m":"T2M"})["T2M"].sel(time = period)
mswx_tmin  = xr.open_dataset("Temperature/Tmin_MSWX_1979_2019d.nc", chunks ="auto").rename({"t2m":"T2M"})["T2M"].sel(time = period)
mswx_tmean = xr.open_dataset("Temperature/Tavg_MSWX_1979_2019d.nc", chunks ="auto").rename({"t2m":"T2M"})["T2M"].sel(time = period)
mswx_lat   = mswx_tmean.lat * np.pi / 180

# W5D5
w5d5_tmax  = xr.open_dataset("Temperature/Tmax_W5E5_1979_2019d.nc", chunks ="auto").rename({"t2m":"T2M"})["T2M"].sel(time = period)
w5d5_tmin  = xr.open_dataset("Temperature/Tmin_W5E5_1979_2019d.nc", chunks ="auto").rename({"t2m":"T2M"})["T2M"].sel(time = period)
w5d5_tmean = xr.open_dataset("Temperature/Tavg_W5E5_1979_2019d.nc", chunks ="auto").rename({"t2m":"T2M"})["T2M"].sel(time = period)
w5d5_lat   = w5d5_tmean.lat * np.pi / 180

## Calculation of Ep

In [ ]:
pet_pmet = pyet.hargreaves(pmet_tmean, pmet_tmax, pmet_tmin, pmet_lat)
pet_pmet = pet_pmet.rename("PET")
pet_pmet.to_netcdf("Evapotranspiration/PET_PMET_1980_2020d.nc", encoding = encode_pet)

pet_pmet_y = pet_pmet.resample(time = "1Y").sum().mean(dim = "time")
pet_pmet_y = pet_pmet_y.where(pet_pmet_y > 0)
pet_pmet_y = pet_pmet_y.rio.write_crs("epsg:4326", inplace=True)
pet_pmet_y.rio.to_raster("Evapotranspiration/PET_PMET_1980_2020.tif")

In [ ]:
pet_cr2met = pyet.hargreaves(cr2met_tmean, cr2met_tmax, cr2met_tmin, cr2met_lat)
pet_cr2met = pet_cr2met.rename("PET")
pet_cr2met.to_netcdf("Evapotranspiration/PET_CR2MET_1980_2020d.nc", encoding = encode_pet)

pet_era5 = pyet.hargreaves(era5_tmean, era5_tmax, era5_tmin, era5_lat)
pet_era5 = pet_era5.rename("PET")
pet_era5.to_netcdf("Evapotranspiration/PET_ERA5_hr_1980_2020d.nc", encoding = encode_pet)

pet_mswx = pyet.hargreaves(mswx_tmean, mswx_tmax, mswx_tmin, mswx_lat)
pet_mswx = pet_mswx.rename("PET")
pet_mswx.to_netcdf("Evapotranspiration/PET_MSWX_1980_2020d.nc", encoding = encode_pet)

pet_w5d5 = pyet.hargreaves(w5d5_tmean, w5d5_tmax, w5d5_tmin, w5d5_lat)
pet_w5d5 = pet_w5d5.rename("PET")
pet_w5d5.to_netcdf("Evapotranspiration/PET_W5E5_1980_2019d.nc", encoding = encode_pet)